# Deep Learning on Fashion MNIST Dataset

## Import standard libraries

## Import preprocessing libraries

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from time import time
from random import randint

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler as ss
from sklearn.decomposition import PCA

import h2o
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [148]:
import scikitplot as skplt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
import sklearn.metrics as metrics

In [9]:
os.chdir("C:\\Users\\Chandrasekaran\\OneDrive - Digital Consulting\\FORE School Mgmt\\Python\\0. Learnings\\data")

In [46]:
data = pd.read_csv("fashion-mnist_train.csv", header=0)

In [84]:
X = data.iloc[:,1:]
y = data.iloc[:,0]

### Data Preprocessing - Normalize the predictors using StandardScaler

In [156]:
se = ss()
X_trans = se.fit_transform(X)
X_trans.shape

(48000, 785)

In [90]:
### Now, split the data
X_train, X_test, y_train,y_test = train_test_split(X_trans,y,test_size=0.2)

In [162]:
y_train = (y_train).reshape(len(y_train),1)
y_train.shape

(48000, 1)

In [163]:
X = np.hstack((X_train, y_train))

In [164]:
X.shape

(48000, 785)

In [165]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,2 hours 5 mins
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.4
H2O cluster version age:,1 month and 7 days
H2O cluster name:,H2O_from_python_Chandrasekaran_g86me5
H2O cluster total nodes:,1
H2O cluster free memory:,1.237 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [ ]:
df = h2o.H2OFrame(X)

In [110]:
df.shape

(48000, 785)

In [114]:
df["C785"]

C785
5
9
1
6
4
4
7
7
4
0


In [118]:
X_columns = df.columns[0:784]

In [121]:
y_columns = df.columns[784]

In [122]:
y_columns

'C785'

In [124]:
df["C785"] = df["C785"].asfactor()

In [126]:
model_h2o = H2ODeepLearningEstimator(
                distribution="multinomial",
                activation="RectifierWithDropout",
                hidden=[50,50,50],
                input_dropout_ratio=0.2,
                standardize=True,
                epochs=500
                )

In [129]:
start = time()
model_h2o.train(X_columns,y_columns,training_frame=df)
end = time()
(end-start)/60

deeplearning Model Build progress: |██████████████████████████████████████| 100%


135.20508241653442

In [131]:
y_test = (y_test.values).reshape(len(y_test),1)

In [132]:
y_test.shape

(12000, 1)

In [133]:
X_test.shape

(12000, 784)

In [135]:
X_test = np.hstack((X_test,y_test))

In [136]:
X_test.shape

(12000, 785)

In [138]:
X_test = h2o.H2OFrame(X_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [139]:
X_test.shape

(12000, 785)

In [141]:
X_test["C785"] = X_test["C785"].asfactor()

In [142]:
result = model_h2o.predict(X_test[:,0:784])

deeplearning prediction progress: |███████████████████████████████████████| 100%


In [144]:
result.as_data_frame().head()

,predict,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9
0,6,3.248586e-03,1.248047e-05,2.024512e-02,1.561082e-03,5.954621e-02,1.887799e-06,9.123831e-01,3.159510e-08,0.003002,7.583083e-09
1,7,5.264756e-07,3.999713e-09,4.719450e-05,5.988773e-06,4.599118e-05,2.544906e-01,1.029072e-04,7.325862e-01,0.003540,9.180174e-03
2,8,6.281952e-14,7.807294e-19,8.674131e-13,1.875360e-15,6.864227e-13,3.093422e-14,6.105197e-10,2.229856e-17,1.000000,5.411223e-16
3,6,4.988251e-03,2.548288e-05,2.622377e-02,2.944915e-03,6.685117e-02,5.609229e-06,8.939217e-01,1.112738e-07,0.005039,2.970252e-08
4,7,1.389168e-07,2.198628e-09,1.421875e-05,1.341952e-06,1.972780e-05,9.612804e-02,3.515702e-05,8.360886e-01,0.001592,6.612074e-02


In [145]:
xe = X_test["C785"].as_data_frame()
xe["result"] = result[0].as_data_frame()
xe.head()

,C785,result
0,6,6
1,7,7
2,8,8
3,6,6
4,7,7


In [146]:
out = (xe["result"] == xe["C785"])
np.sum(out)/out.size

0.8620833333333333